# DAT210x - Programming with Python for DS

## Module5- Lab5

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.decomposition import PCA

matplotlib.style.use('ggplot') # Look Pretty

### A Convenience Function

In [2]:
def plotDecisionBoundary(model, X, y):
    fig = plt.figure()
    ax = fig.add_subplot(111)

    padding = 0.6
    resolution = 0.0025
    colors = ['royalblue','forestgreen','ghostwhite']

    # Calculate the boundaris
    x_min, x_max = X[:, 0].min(), X[:, 0].max()
    y_min, y_max = X[:, 1].min(), X[:, 1].max()
    x_range = x_max - x_min
    y_range = y_max - y_min
    x_min -= x_range * padding
    y_min -= y_range * padding
    x_max += x_range * padding
    y_max += y_range * padding

    # Create a 2D Grid Matrix. The values stored in the matrix
    # are the predictions of the class at at said location
    xx, yy = np.meshgrid(np.arange(x_min, x_max, resolution),
                       np.arange(y_min, y_max, resolution))

    # What class does the classifier say?
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # Plot the contour map
    cs = plt.contourf(xx, yy, Z, cmap=plt.cm.terrain)

    # Plot the test original points as well...
    for label in range(len(np.unique(y))):
        indices = np.where(y == label)
        plt.scatter(X[indices, 0], X[indices, 1], c=colors[label], label=str(label), alpha=0.8)

    p = model.get_params()
    plt.axis('tight')
    plt.title('K = ' + str(p['n_neighbors']))

### The Assignment

Load up the dataset into a variable called `X`. Check `.head` and `dtypes` to make sure you're loading your data properly--don't fail on the 1st step!

In [48]:
# .. your code here ..
X=pd.read_csv('Datasets/wheat.data')
X.columns

Index([u'id', u'area', u'perimeter', u'compactness', u'length', u'width',
       u'asymmetry', u'groove', u'wheat_type'],
      dtype='object')

Copy the `wheat_type` series slice out of `X`, and into a series called `y`. Then drop the original `wheat_type` column from the `X`:

In [49]:
# .. your code here ..
y=X['wheat_type'].copy()
#X=pd.read_csv('Datasets/wheat.data')
#X.drop(X.iloc[:0],axis=0)
X.drop(labels=['wheat_type'],inplace=True,axis=1)
X.drop(labels=['id'],inplace=True,axis=1)
#X.drop(X.iloc[0],axis=0)
#X.drop(labels=[0],axis=1,inplace=True)
#X

Do a quick, "ordinal" conversion of `y`. In actuality our classification isn't ordinal, but just as an experiment...

In [50]:
# .. your code here ..

#y[0].apply(lambda x: x.toordinal())
#y['wheat_type'].apply(object.toordinal())
y=y.astype('category').cat.codes

Do some basic nan munging. Fill each row's nans with the mean of the feature:

In [51]:
# .. your code here ..
X.fillna(X.mean(),inplace=True)

,area,perimeter,compactness,length,width,asymmetry,groove
0,15.26,14.84,0.8710,5.763,3.312,2.2210,5.220000
1,14.88,14.57,0.8811,5.554,3.333,1.0180,4.956000
2,14.29,14.09,0.9050,5.291,3.337,2.6990,4.825000
3,13.84,13.94,0.8955,5.324,3.379,2.2590,4.805000
4,16.14,14.99,0.9034,5.658,3.562,1.3550,5.175000
5,14.38,14.21,0.8951,5.386,3.312,2.4620,4.956000
6,14.69,14.49,0.8799,5.563,3.259,3.5860,5.219000
7,14.11,14.10,0.8911,5.420,3.302,2.7000,5.407529
8,16.63,15.46,0.8747,6.053,3.465,2.0400,5.877000
9,16.44,15.25,0.8880,5.884,3.505,1.9690,5.533000


Split `X` into training and testing data sets using `train_test_split()`. Use `0.33` test size, and use `random_state=1`. This is important so that your answers are verifiable. In the real world, you wouldn't specify a random_state:

In [71]:
# .. your code here ..
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.33, random_state=1)
y_train
X_train

,area,perimeter,compactness,length,width,asymmetry,groove
147,12.49,13.46,0.86580,5.2670,2.967,4.421,5.002000
78,18.94,16.49,0.87500,6.4450,3.639,5.064,6.362000
117,19.15,16.45,0.88900,6.2450,3.815,3.084,6.185000
42,13.16,13.55,0.90090,5.1380,3.201,2.461,4.783000
17,15.69,14.75,0.90580,5.5270,3.514,1.599,5.046000
5,14.38,14.21,0.89510,5.3860,3.312,2.462,4.956000
154,11.36,13.05,0.83820,5.1750,2.755,4.048,5.263000
171,11.55,13.10,0.84550,5.1670,2.845,6.715,4.956000
48,14.79,14.52,0.88190,5.5450,3.291,2.704,5.111000
169,11.24,13.00,0.87128,0.8359,5.090,2.715,3.521000


Create an instance of SKLearn's Normalizer class and then train it using its .fit() method against your _training_ data. The reason you only fit against your training data is because in a real-world situation, you'll only have your training data to train with! In this lab setting, you have both train+test data; but in the wild, you'll only have your training data, and then unlabeled data you want to apply your models to.

In [72]:
# .. your code here ..
T = preprocessing.Normalizer()
#model = KNeighborsClassifier(n_neighbors=3)
norm_fit=T.fit(X_train) 
norm_fit


Normalizer(copy=True, norm='l2')

With your trained pre-processor, transform both your training AND testing data. Any testing data has to be transformed with your preprocessor that has ben fit against your training data, so that it exist in the same feature-space as the original data used to train your models.

In [73]:
# .. your code here ..
norm_train=T.transform(X_train)
norm_test=T.transform(X_test)


Just like your preprocessing transformation, create a PCA transformation as well. Fit it against your training data, and then project your training and testing features into PCA space using the PCA model's `.transform()` method. This has to be done because the only way to visualize the decision boundary in 2D would be if your KNN algo ran in 2D as well:

In [74]:
# .. your code here ..
dimensions=2
model = PCA(n_components=dimensions, svd_solver='randomized', random_state=1)
pca_fit=model.fit(X_train)
pca_train=model.transform(norm_train)
pca_test=model.transform(norm_test)


Create and train a KNeighborsClassifier. Start with `K=9` neighbors. Be sure train your classifier against the pre-processed, PCA- transformed training data above! You do not, of course, need to transform your labels.

In [93]:
# .. your code here ..
knn= KNeighborsClassifier(n_neighbors=1)
knn.fit(pca_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [94]:
# I hope your KNeighbors classifier model from earlier was named 'knn'
# If not, adjust the following line:
plotDecisionBoundary(knn, pca_train, y_train)

Display the accuracy score of your test data/labels, computed by your KNeighbors model. You do NOT have to run `.predict` before calling `.score`, since `.score` will take care of running your predictions for you automatically.

In [95]:
# .. your code here ..
knn.score(pca_test,y_test)

0.84285714285714286

### Bonus

Instead of the ordinal conversion, try and get this assignment working with a proper Pandas get_dummies for feature encoding. You might have to update some of the `plotDecisionBoundary()` code.

In [ ]:
plt.show()